## 使用DeepSeek进行数据表的查询

In [2]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor

db_user = "student123"
db_password = "student321"
#db_host = "localhost:3306"
db_host = "rm-uf6z891lon6dxuqblqo.mysql.rds.aliyuncs.com:3306"
db_name = "action"
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")
db

In [3]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    temperature=0.01,
    model="deepseek-chat",  
    openai_api_key = "sk-9846f14a2104490b960adbf5c5b3b32e",
    openai_api_base="https://api.deepseek.com"
    # openai_api_key ="sk-da635dce04da45779b76d549568126f0"
)

# 需要设置llm
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_23512\2770502453.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [4]:
# Task: 描述数据表
agent_executor.run("描述与订单相关的表及其关系")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_23512\2182499378.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent_executor.run("描述与订单相关的表及其关系")




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""address, asset_grades, bank, car_sales, customers, dept, employee, form, height_grades, hero_score, heros, orders, person, player, player_score, student, team, team_score, test_work, trips, user, users, using, weatherI see that there is an "orders" table which is likely related to orders. I should also check for other tables that might be related to orders, such as "customers" or "users" since orders are typically associated with customers/users. I'll query the schema for these tables to understand their structure and relationships.

Action: sql_db_schema
Action Input: "orders, customers, users"
CREATE TABLE customers (
	`Id` INTEGER, 
	`Name` VARCHAR(255)
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb3

/*
3 rows from customers table:
Id	Name
1	Joe
2	Henry
3	Sam
*/


CREATE TABLE orders (
	`Id` INTEGER, 
	`CustomerId` INTEGER
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb3

/*
3 rows from orders table:
Id	CustomerId

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `From the schema I can see that:

1. The main order-related tables are:
   - `orders` table: Contains order IDs and references customer IDs
   - `customers` table: Contains customer IDs and names
   - `users` table: Contains user information including banned status and role (though this doesn't appear directly related to orders)

2. The relationships are:
   - The `orders` table has a foreign key relationship with the `customers` table through the `CustomerId` field which references the `Id` field in the `customers` table
   - Each order is associated with exactly one customer (one-to-many relationship where one customer can have many orders)
   - There doesn't appear to be a direct relationship between `orders` and `users` tables based on the schema shown

3. Sample data

In [ ]:
# 这个任务，实际上数据库中 没有HeroDetails表
agent_executor.run("描述HeroDetails表")

In [ ]:
agent_executor.run("描述Hero表")

In [ ]:
agent_executor.run("找出英雄攻击力最高的前5个英雄")